In [41]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [42]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [ ]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.keras')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1321217 (5.04 MB)
Trainable params: 1321217 (5.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.get_weights()

[array([[ 0.05887557,  0.06234861,  0.02030891, ...,  0.09033559,
          0.00102111,  0.02626594],
        [ 0.00321989, -0.07369632, -0.10887146, ...,  0.03386511,
          0.01121884,  0.08798378],
        [-0.09053239,  0.0476514 , -0.00321312, ..., -0.12581113,
          0.02614552,  0.00030044],
        ...,
        [-0.00181175,  0.02548836, -0.04564215, ...,  0.1399298 ,
          0.02057713, -0.14009668],
        [-0.01838053,  0.05165372,  0.06171583, ..., -0.06167265,
          0.06396143,  0.07543787],
        [ 0.02581083,  0.09082711, -0.16409402, ...,  0.13268404,
         -0.07584815, -0.24145158]], dtype=float32),
 array([[-0.09008233, -0.19279705,  0.01578155, ..., -0.04453309,
          0.04625545, -0.0107862 ],
        [-0.3236066 , -0.02815419, -0.05847537, ...,  0.0610173 ,
          0.15290111, -0.23859087],
        [-0.05654524, -0.02842003, -0.0498878 , ..., -0.14307526,
         -0.0245651 ,  0.05955515],
        ...,
        [-0.01609726, -0.19937666, -0.1

In [47]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    # Clean and tokenize text
    text = text.lower()
    # Remove punctuation except apostrophes
    text = ''.join([c for c in text if c.isalnum() or c.isspace() or c=="'"])
    words = text.split()
    
    # Convert words to indices with proper offset
    encoded_review = []
    for word in words:
        # Get word index and add offset of 3 as per IMDB dataset
        idx = word_index.get(word)
        if idx is not None:
            encoded_review.append(idx + 3)
        else:
            encoded_review.append(2)  # Unknown token
    
    # Pad sequence
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

def predict_sentiment(review, threshold=0.5):
    # Preprocess the input
    preprocessed_input = preprocess_text(review)
    
    # Get prediction
    prediction = model.predict(preprocessed_input, verbose=0)[0][0]
    
    # Calculate confidence
    confidence = abs(prediction - 0.5) * 2  # Scale to 0-100%
    
    sentiment = 'Positive' if prediction > threshold else 'Negative'
    
    return {
        'sentiment': sentiment,
        'score': float(prediction),
        'confidence': float(confidence * 100)
    }

# Test with multiple examples
test_reviews = [
    "This movie was absolutely fantastic! Great acting and amazing plot.",
    "I didn't like this movie at all. Poor direction and boring story.",
    "The movie was okay, not great but not terrible either.",
    "The movie was a waste of time. I want my money back!"
]

for review in test_reviews:
    result = predict_sentiment(review)
    print(f"\nReview: {review}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Score: {result['score']:.3f}")
    print(f"Confidence: {result['confidence']:.1f}%")



Review: This movie was absolutely fantastic! Great acting and amazing plot.
Sentiment: Positive
Score: 0.841
Confidence: 68.3%

Review: I didn't like this movie at all. Poor direction and boring story.
Sentiment: Negative
Score: 0.035
Confidence: 92.9%

Review: The movie was okay, not great but not terrible either.
Sentiment: Negative
Score: 0.089
Confidence: 82.2%

Review: The movie was a waste of time. I want my money back!
Sentiment: Negative
Score: 0.010
Confidence: 98.0%
